In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

2024-11-15 15:15:07.958743: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 15:15:08.086769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 15:15:08.086922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 15:15:08.105976: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 15:15:08.143227: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from module import Ranger

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
# gpu settings

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("GPU detected: ", gpus[0])
    try:
        tf.config.set_visible_devices(gpus[0], "GPU")
        ldev = tf.config.list_logical_devices("GPU")
        print(len(ldev), "Logical devices and ", len(gpus), "Physical devices detected")
    except RuntimeError as e:
        print(e)

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except:
        pass

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
print(os.getenv("TF_GPU_ALLOCATOR"))

tf.debugging.set_log_device_placement(True)

GPU detected:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Logical devices and  1 Physical devices detected
cuda_malloc_async


2024-11-15 15:15:10.520438: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 15:15:10.552505: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-15 15:15:10.552668: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
train_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/train_synsets"
valid_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"
test_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/test_synsets"

model_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Models"

In [6]:
num_epochs = 14
batch_size = 32
img_h = 224
img_w = 224
num_channels = 3
num_classes = 3

In [7]:
class MyNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            32,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            input_shape=input_shape,
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(5, 5),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(5, 5),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        # self.add(BatchNormalization(momentum=1))
        self.add(Flatten())
        # self.add(Dropout(0.5))
        # self.add(Dense(4096, activation="relu"))
        self.add(Dropout(0.4))
        # self.add(Dense(3796, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            # optimizer=Ranger(learning_rate=0.0001),
            # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
            optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )


In [8]:
model = MyNet((img_w, img_h, num_channels), num_classes)
model.summary()

Model: "my_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 128)     36992     
                                                                 
 max_pooling2d (MaxPooling2  (None, 54, 54, 128)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 256)       295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 256)       590080    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 24, 24, 256)       0         
 g2D)                                                       

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=1,
    height_shift_range=1,
    shear_range=0.6,
    zoom_range=1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples
train_num = train_generator.samples

Found 5688 images belonging to 5 classes.
Found 505 images belonging to 5 classes.
Found 283 images belonging to 5 classes.


In [10]:
logs_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
callbacks_list = [tensorboard_callbacks]

model.fit(
    train_generator,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    verbose=1
)

2024-11-15 15:15:11.299791: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/14


2024-11-15 15:15:12.669778: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-11-15 15:15:12.784452: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-15 15:15:17.222084: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-15 15:15:18.163869: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/abhinav/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/abhinav/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/abhinav/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/abhinav/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/abhinav/.local/lib/python3.10/si

In [ ]:
model.evaluate(
    test_generator,
    verbose=2
)

In [ ]:
# model.save(os.path.join(model_dir, "Mynet1"))